In [3]:
import matplotlib.pyplot as plt
import cv2, re
import numpy as np
from queue import PriorityQueue

In [147]:
# BMP to JPEG conversion
img = cv2.imread('snail.bmp')

s = img.shape
new_shape = (((s[0] + 15) // 16) * 16, ((s[1] + 15) // 16) * 16, s[2])
new_image = np.zeros(new_shape, dtype = 'float32')
new_image[:s[0], :s[1], :] = img
img = np.array(new_image)
old_shape = s
s = new_shape
s1 = (s[0] // 2, s[1] // 2)

# todo - padding

# conversion from RGB format to YCbCr (Y -> Luminance, Cb -> blue chrominance, Cr -> red chrominance)
imgYCC = cv2.cvtColor(img, cv2.COLOR_BGR2YCR_CB)
Y = imgYCC[:,:,0]
Cb = np.zeros(s1, dtype = int)
Cr = np.zeros(s1, dtype = int)

# average every 2x2 block for Cb and Cr
for i in range(s[0]//2):
    for j in range(s[1]//2):
        x = i*2
        y = j*2
        avg = (imgYCC[x, y, 1] + imgYCC[x, y + 1, 1] + imgYCC[x + 1, y, 1] + imgYCC[x + 1, y + 1, 1]) // 4
        Cr[i, j] = avg
        avg = (imgYCC[x][y][2] + imgYCC[x][y + 1][2] + imgYCC[x + 1][y][2] + imgYCC[x + 1][y + 1][2]) // 4
        Cb[i, j] = avg

# Discrete Cosine Transform
Y -= 128
Cb -= 128
Cr -= 128
Y_blocks = []
Cb_blocks = []
Cr_blocks = []
for i in range(0, s[0], 8):
    for j in range(0, s[1], 8):
        grid = np.float32(Y[i:i+8, j:j+8])
        Y_blocks.append(cv2.dct(grid))
for i in range(0, s1[0], 8):
    for j in range(0, s1[1], 8):
        grid = np.float32(Cb[i:i+8, j:j+8])
        Cb_blocks.append(cv2.dct(grid))
        grid = np.float32(Cr[i:i+8, j:j+8])
        Cr_blocks.append(cv2.dct(grid))

len_y = len(Y_blocks)
len_c = len(Cb_blocks)

# Quantization
Y_q = np.array([[4., 3, 4, 4, 4, 6, 11, 15], [3, 3, 3, 4, 5, 8, 14, 19], [3, 4, 4, 5, 8, 12, 16, 20], [4, 5, 6, 7, 12, 14, 18, 20], [6, 6, 9, 11, 14, 17, 21, 23], [9, 12, 12, 18, 23, 22, 25, 21], [11, 13, 15, 17, 21, 23, 25, 21], [13, 12, 12, 13, 16, 19, 21, 21]])
C_q = np.array([[4., 4, 6, 10, 21, 21, 21, 21], [4, 5, 6, 21, 21, 21, 21, 21], [6, 6, 12, 21, 21, 21, 21, 21], [10, 14, 21, 21, 21, 21, 21, 21], [21, 21, 21, 21, 21, 21, 21, 21], [21, 21, 21, 21, 21, 21, 21, 21], [21, 21, 21, 21, 21, 21, 21, 21], [21, 21, 21, 21, 21, 21, 21, 21]])
for i in range(len_y):
    Y_blocks[i] //= Y_q
for i in range(len_c):
    Cb_blocks[i] //= C_q
for i in range(len_c):
    Cr_blocks[i] //= C_q

# Run length encoding
def spiral_traversal(block):
    p = 8
    arr = np.array([])
    k = 0
    for c in range(p):
        for i in range(0, k+1):
            if k%2!=0 :
                arr = np.append(arr, block[i][k-i])
            else :
                arr = np.append(arr, block[k-i][i])
        k+=1

    k = 1
    for c in range(p-1):
        j = p
        for i in range(k, p):
            j-=1
            if k%2 == 0:
                arr = np.append(arr, block[i][j])
            else:
                arr = np.append(arr, block[j][i])
        k+=1
    return arr

dc = 0
def encode(block):
    global dc
    temp = spiral_traversal(block)
    l = len(temp)

    encoded_block = []
    encoded_block.append((0, temp[0] - dc))
    dc = temp[0]
    c0 = 0
    for i in range(1, l):
        if temp[i] or c0==15 :
            encoded_block.append((c0, temp[i]))
            c0 = 0
        else:
            c0 += 1
    encoded_block.append((0, 0))

    encoded_block = np.array(encoded_block, dtype = int)
    return encoded_block

encoded_Y = list()
encoded_Cr = list()
encoded_Cb = list()
for i in range(len_y):
    encoded_Y.append(encode(Y_blocks[i]))
for i in range(len_c):
    encoded_Cb.append(encode(Cb_blocks[i]))
    encoded_Cr.append(encode(Cr_blocks[i]))
encoded_Y = np.array(encoded_Y, dtype = object)
encoded_Cr = np.array(encoded_Cr, dtype = object)
encoded_Cb = np.array(encoded_Cb, dtype = object)

# print(encoded_Y, encoded_Cb, encoded_Cr, sep = '\n')

# Huffman Tables
def get_cat(num): ##extracting position of msb to determine what length of bits it will need to be encoded
    num = int(abs(num))
    ans = 0
    pwr = 1
    while pwr < num:
        pwr <<= 1
        ans += 1
    return ans
##***********************************************************DANGER**********************


def make_freq_table(dc_freq, ac_freq, encoded_blocks):
    for block in encoded_blocks:
        #block of length 64 usually (we take 8*8 grids), first element is dc coeff, the rest are ac coeffs
        #dc block[0] is encoded as (0, value)
        dc = block[0]
        dc_run_length, dc_val = dc #dc_run_length will be 0
        
        cat = get_cat(dc_val)
        dc_freq[dc_run_length, cat] += 1
        
        for i in range(1, len(block)):
            
            #ac block[i] is encoded as (run length of zeros before value max 15, value of ith non zero ac coefficient)
            ac_i = block[i]
            run_length, value = ac_i
            cat = get_cat(value) # find the category of value
            ac_freq[run_length, cat] += 1
            
dc_huffman_Y = np.zeros((1, 16), dtype=int)
ac_huffman_Y = np.zeros((16,16), dtype=int)
dc_huffman_C = np.zeros((1, 16), dtype=int)
ac_huffman_C = np.zeros((16,16), dtype=int)

make_freq_table(dc_huffman_Y, ac_huffman_Y, encoded_Y)
make_freq_table(dc_huffman_C, ac_huffman_C, encoded_Cr)
make_freq_table(dc_huffman_C, ac_huffman_C, encoded_Cb)



# make huffman codes
depth = 0
def get_codebook(freq):
    codebook = np.empty(freq.shape, dtype = object)
    def get_code_lengths(freq):

        def make_tree(freq):
            global code
            q = PriorityQueue()
            m = len(freq)
            n = len(freq[0])

            cnt = 0
            for i in range(m):
                for j in range(n):
                    if freq[i,j]:
                        q.put((freq[i,j], str((i,j))))
                        cnt += 1

            for i in range(cnt-1):
                a = q.get()
                b = q.get()
                q.put((a[0]+b[0], str("[" + a[1] + "," + b[1] + "]")))

            tree = eval(q.get()[1])
            return tree

        tree = make_tree(freq)
        code_lengths = np.zeros(freq.shape, dtype=int)

        depth = 0
        def get_depths(arr, code_lengths):
            global depth
            if type(arr[1]) == int and type(arr[0]) == int:
                code_lengths[arr[0], arr[1]] = min(depth, 16)
            else:
                depth += 1
                get_depths(arr[0], code_lengths)
                get_depths(arr[1], code_lengths)
                depth-=1

        get_depths(tree, code_lengths)
        return code_lengths
    
    code_lengths = get_code_lengths(freq)
    length_symbol_pairs = []
    
    m, n = freq.shape
    for i in range(m):
        for j in range(n):
            if code_lengths[i, j]:
                length_symbol_pairs.append((code_lengths[i, j], (i, j)))
    length_symbol_pairs.sort()
    
    def get_code(code, length):
        st = bin(code).replace("0b", "")[::-1]
        while len(st) < length:
            st += '0'
        return st[::-1]
    
    code = 0
    cur_len = 0
    for length, symbol in length_symbol_pairs:
        while cur_len < length:  #if there is no symbol with the current length
            cur_len += 1
            code *= 2
        codebook[symbol[0], symbol[1]] = get_code(code, length)
        code += 1
    
    return codebook, length_symbol_pairs

codes_dc_Y, sorted_symbols_dc_Y = get_codebook(dc_huffman_Y)
codes_ac_Y, sorted_symbols_ac_Y = get_codebook(ac_huffman_Y)
codes_dc_C, sorted_symbols_dc_C = get_codebook(dc_huffman_C)
codes_ac_C, sorted_symbols_ac_C = get_codebook(ac_huffman_C)

def to_two_bytes(num):
    bh = bin(num).replace("0b", "")
    bh = bh[::-1]
    while len(bh) < 16:
        bh += '0'
    bh = bh[::-1]

    return [int(bh[:8], 2), int(bh[8:], 2)]

# Writing jpeg image in binary
jpeg_image = [0xff, 0xd8, 0xff, 0xe0, 0x00, 0x10]

header = [0x4a, 0x46, 0x49, 0x46, 0x00, 0x01, 0x01, 0x01, 0x00, 0x48, 0x00, 0x48, 0x00, 0x00]
jpeg_image.extend(header)

luminance_quantisation_table = [0xff, 0xdb, 0x00, 0x43, 0x00]
luminance_quantisation_table.extend([int(ele) for ele in spiral_traversal(Y_q)])
jpeg_image.extend(luminance_quantisation_table)

chrominance_quantisation_table = [0xff, 0xdb, 0x00, 0x43, 0x01]
chrominance_quantisation_table.extend([int(ele) for ele in spiral_traversal(C_q)])
jpeg_image.extend(chrominance_quantisation_table)

start_of_frame = [0xff, 0xc0, 0x00, 0x11, 0x08]
start_of_frame.extend(to_two_bytes(old_shape[0]))
start_of_frame.extend(to_two_bytes(old_shape[1]))
start_of_frame.extend([0x03, 0x01, 0x22, 0x00, 0x02, 0x11, 0x01, 0x03, 0x11, 0x01])
jpeg_image.extend(start_of_frame)

def parse_table(sorted_symbols, typ):
    table = [0xff, 0xc4]
    ln = 19
    symbols = []
    freqs = [0 for i in range(17)]
    
    for length, symbol in sorted_symbols:
        freqs[length] += 1
        symbols.append(symbol[0] * 16 + symbol[1])
        ln += 1
    
    table.extend(to_two_bytes(ln))
    table.append(typ)
    table.extend(freqs)
    table.extend(symbols)
    return table

jpeg_image.extend(parse_table(sorted_symbols_dc_Y, 0x00))
jpeg_image.extend(parse_table(sorted_symbols_ac_Y, 0x10))
jpeg_image.extend(parse_table(sorted_symbols_dc_C, 0x01))
jpeg_image.extend(parse_table(sorted_symbols_ac_C, 0x11))

start_of_scan = [0xff, 0xda, 0x00, 0x0c, 0x03, 0x01, 0x00, 0x02, 0x11, 0x03, 0x11, 0x00, 0x3f, 0x00]
jpeg_image.extend(start_of_scan)

def bit_rep(value):
    if value == 0:
        return ''
    else : 
        v = bin(abs(value)).replace("0b", '')
        if value>0 :
            return v
        else:
            return v.replace('0', 'a').replace('1','0').replace('a','1')

def encode_data(encoded_Y, encoded_Cb, encoded_Cr, codes_dc_Y, code_dc_C, codes_ac_Y, codes_ac_C):
    s = ''
    def encode_block(block, dc_codebook, ac_codebook):
        ans = ''
        dc_val = block[0][1]
        cat = get_cat(dc_val)
        ans+=(dc_codebook[0,cat]+bit_rep(dc_val))
        
        for i in range(1, len(block)):
            run, val = block[i]
            ans+=(ac_codebook[run, get_cat(val)]+bit_rep(val))
        return ans
    
    for i in range(len(encoded_Y)):
        s += encode_block(encoded_Y[i], codes_dc_Y, codes_ac_Y)
        s += encode_block(encoded_Cb[i // 4], codes_dc_C, codes_ac_C)
        s += encode_block(encoded_Cr[i // 4], codes_dc_C, codes_ac_C)
        
    #pad with 0s
    a = len(s)%8
    s+='0'*a
    
    data = [int(s[i:i+8], 2) for i in range(0, len(s), 8)]
    fin_data = []
    for ele in data:
        fin_data.append(ele)
        if ele == 0xff:
            fin_data.append(0x00)
    return fin_data
    
jpeg_image.extend(encode_data(encoded_Y, encoded_Cb, encoded_Cr, codes_dc_Y, codes_dc_C, codes_ac_Y, codes_ac_C))

end_of_image = [0xff, 0xd9]
jpeg_image.extend(end_of_image)

try:
    with open("my_image.jpeg", 'wb') as f:
        for byte in jpeg_image:
            f.write(byte.to_bytes(1, byteorder='big'))
except Exception as e:
    print(e)

file = open('my_image.jpeg', 'rb')
data = file.read()
s = data.hex()

l = re.findall('..?', s)
d = np.reshape(l, (-1,1))
d = [' '.join(ele) for ele in d]
for ele in d:
    print(ele)
file.close()

ff
d8
ff
e0
00
10
4a
46
49
46
00
01
01
01
00
48
00
48
00
00
ff
db
00
43
00
04
03
03
03
03
04
04
03
04
04
06
05
04
04
04
06
05
05
06
06
09
0b
0c
09
07
08
08
0b
0f
0e
0c
0c
0b
0c
0d
0d
0c
0f
12
0e
0e
10
13
14
12
11
17
11
0c
0d
15
16
15
14
17
19
17
10
13
19
15
15
15
15
ff
db
00
43
01
04
04
04
06
05
06
0a
06
06
0a
15
0e
0c
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
ff
c0
00
11
08
01
00
01
00
03
01
22
00
02
11
01
03
11
01
ff
c4
00
1d
00
00
01
00
02
03
01
01
02
00
00
00
00
00
00
00
00
00
00
05
06
03
04
07
08
02
01
09
ff
c4
00
40
10
00
00
01
03
03
03
03
03
03
01
06
02
07
0a
00
00
00
00
01
02
03
04
05
10
06
11
20
07
12
30
13
21
40
08
14
22
50
15
23
31
32
60
70
16
41
24
25
26
33
42
71
80
17
18
34
35
51
52
61
a0
c0
f0
ff
c4
00
1d
01
00
01
00
02
03
01
01
02
00
00
00
00
00
00
00
00
00
00
07
08
04
05
06
03
01
02
09
ff
c4
00
34
11
00
01
00
01
03
03
02
05
01
07
04
02
04
00
00
00
00
00
10
01
0

In [100]:
file = open('snail.jpeg', 'rb')
data = file.read()
s = data.hex()

import re
l = re.findall('..?', s)
print(len(l))
d = np.reshape(l, (-1, 11))
d = [' '.join(ele) for ele in d]
for ele in d:
    print(ele)
file.close()

bit_rep(-63)

13211
ff d8 ff e0 00 10 4a 46 49 46 00
01 01 01 00 60 00 60 00 00 ff db
00 43 00 03 02 02 03 02 02 03 03
03 03 04 03 03 04 05 08 05 05 04
04 05 0a 07 07 06 08 0c 0a 0c 0c
0b 0a 0b 0b 0d 0e 12 10 0d 0e 11
0e 0b 0b 10 16 10 11 13 14 15 15
15 0c 0f 17 18 16 14 18 12 14 15
14 ff db 00 43 01 03 04 04 05 04
05 09 05 05 09 14 0d 0b 0d 14 14
14 14 14 14 14 14 14 14 14 14 14
14 14 14 14 14 14 14 14 14 14 14
14 14 14 14 14 14 14 14 14 14 14
14 14 14 14 14 14 14 14 14 14 14
14 14 14 14 ff c0 00 11 08 01 00
01 00 03 01 22 00 02 11 01 03 11
01 ff c4 00 1f 00 00 01 05 01 01
01 01 01 01 00 00 00 00 00 00 00
00 01 02 03 04 05 06 07 08 09 0a
0b ff c4 00 b5 10 00 02 01 03 03
02 04 03 05 05 04 04 00 00 01 7d
01 02 03 00 04 11 05 12 21 31 41
06 13 51 61 07 22 71 14 32 81 91
a1 08 23 42 b1 c1 15 52 d1 f0 24
33 62 72 82 09 0a 16 17 18 19 1a
25 26 27 28 29 2a 34 35 36 37 38
39 3a 43 44 45 46 47 48 49 4a 53
54 55 56 57 58 59 5a 63 64 65 66
67 68 69 6a 73 74 75 76 77 78 79
7a 83 84 85 86 87 88 89 8a 92 93
94 9

'000000'

In [80]:
# BMP to JPEG conversion
img = cv2.imread('snail.bmp')

s = img.shape
new_shape = (((s[0] + 15) // 16) * 16, ((s[1] + 15) // 16) * 16, s[2])
new_image = np.zeros(new_shape, dtype = 'float32')
new_image[:s[0], :s[1], :] = img
img = np.array(new_image)
old_shape = s
s = new_shape
s1 = (s[0] // 2, s[1] // 2)

# todo - padding

# conversion from RGB format to YCbCr (Y -> Luminance, Cb -> blue chrominance, Cr -> red chrominance)
imgYCC = cv2.cvtColor(img, cv2.COLOR_BGR2YCR_CB)
Y = imgYCC[:,:,0]
Cb = np.zeros(s1, dtype = int)
Cr = np.zeros(s1, dtype = int)

# average every 2x2 block for Cb and Cr
for i in range(s[0]//2):
    for j in range(s[1]//2):
        x = i*2
        y = j*2
        avg = (imgYCC[x, y, 1] + imgYCC[x, y + 1, 1] + imgYCC[x + 1, y, 1] + imgYCC[x + 1, y + 1, 1]) // 4
        Cr[i, j] = avg
        avg = (imgYCC[x][y][2] + imgYCC[x][y + 1][2] + imgYCC[x + 1][y][2] + imgYCC[x + 1][y + 1][2]) // 4
        Cb[i, j] = avg

# Discrete Cosine Transform
Y -= 128
Cb -= 128
Cr -= 128
Y_blocks = []
Cb_blocks = []
Cr_blocks = []
for i in range(0, s[0], 8):
    for j in range(0, s[1], 8):
        grid = np.float32(Y[i:i+8, j:j+8])
        Y_blocks.append(cv2.dct(grid))
for i in range(0, s1[0], 8):
    for j in range(0, s1[1], 8):
        grid = np.float32(Cb[i:i+8, j:j+8])
        Cb_blocks.append(cv2.dct(grid))
        grid = np.float32(Cr[i:i+8, j:j+8])
        Cr_blocks.append(cv2.dct(grid))

len_y = len(Y_blocks)
len_c = len(Cb_blocks)

def spiralo(block):
    p = 8
    arr = []
    k = 0
    for c in range(p):
        for i in range(0, k+1):
            if k%2!=0 :
                arr.append(block[i][k-i])
            else :
                arr.append(block[k-i][i])
        k+=1

    k = 1
    for c in range(p-1):
        j = p
        for i in range(k, p):
            j-=1
            if k%2 == 0:
                arr.append(block[i][j])
            else:
                arr.append(block[j][i])
        k+=1
    return arr

# Quantization
myArr = [
[0x03, 0x02, 0x02, 0x03, 0x02, 0x02, 0x03, 0x03],
[0x03, 0x03, 0x04, 0x03, 0x03, 0x04, 0x05, 0x08],
[0x05, 0x05, 0x04, 0x04, 0x05, 0x0a, 0x07, 0x07],
[0x06, 0x08, 0x0c, 0x0a, 0x0c, 0x0c, 0x0b, 0x0a],
[0x0b, 0x0b, 0x0d, 0x0e, 0x12, 0x10, 0x0d, 0x0e],
[0x11, 0x0e, 0x0b, 0x0b, 0x10, 0x16, 0x10, 0x11],
[0x13, 0x14, 0x15, 0x15, 0x15, 0x0c, 0x0f, 0x17],
[0x18, 0x16, 0x14, 0x18, 0x12, 0x14, 0x15, 0x14]
]
myArr2 = [
[0x03, 0x04, 0x04, 0x05, 0x04, 0x05, 0x09, 0x05],
[0x05, 0x09, 0x14, 0x0d, 0x0b, 0x0d, 0x14, 0x14],
[0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14],
[0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14],
[0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14],
[0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14],
[0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14],
[0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14]
]
spiral = [[(i, j) for j in range(8)] for i in range(8)]
spiral = spiralo(spiral)
Y_q = np.array([[4., 3, 4, 4, 4, 6, 11, 15], [3, 3, 3, 4, 5, 8, 14, 19], [3, 4, 4, 5, 8, 12, 16, 20], [4, 5, 6, 7, 12, 14, 18, 20], [6, 6, 9, 11, 14, 17, 21, 23], [9, 12, 12, 18, 23, 22, 25, 21], [11, 13, 15, 17, 21, 23, 25, 21], [13, 12, 12, 13, 16, 19, 21, 21]])
C_q = np.array([[4., 4, 6, 10, 21, 21, 21, 21], [4, 5, 6, 21, 21, 21, 21, 21], [6, 6, 12, 21, 21, 21, 21, 21], [10, 14, 21, 21, 21, 21, 21, 21], [21, 21, 21, 21, 21, 21, 21, 21], [21, 21, 21, 21, 21, 21, 21, 21], [21, 21, 21, 21, 21, 21, 21, 21], [21, 21, 21, 21, 21, 21, 21, 21]])
cur = 0
for i in range(8):
    for j in range(8):
        Y_q[spiral[cur][0]][spiral[cur][1]] = myArr[i][j]
        C_q[spiral[cur][0]][spiral[cur][1]] = myArr2[i][j]
        cur += 1
        
for i in range(len_y):
    Y_blocks[i] //= Y_q
for i in range(len_c):
    Cb_blocks[i] //= C_q
for i in range(len_c):
    Cr_blocks[i] //= C_q

# Run length encoding
def spiral_traversal(block):
    p = 8
    arr = np.array([])
    k = 0
    for c in range(p):
        for i in range(0, k+1):
            if k%2!=0 :
                arr = np.append(arr, block[i][k-i])
            else :
                arr = np.append(arr, block[k-i][i])
        k+=1

    k = 1
    for c in range(p-1):
        j = p
        for i in range(k, p):
            j-=1
            if k%2 == 0:
                arr = np.append(arr, block[i][j])
            else:
                arr = np.append(arr, block[j][i])
        k+=1
    return arr

dc = 0
def encode(block):
    global dc
    temp = spiral_traversal(block)
    l = len(temp)

    encoded_block = []
    encoded_block.append((0, temp[0] - dc))
    dc = temp[0]
    c0 = 0
    for i in range(1, l):
        if temp[i] or c0==15 :
            encoded_block.append((c0, temp[i]))
            c0 = 0
        else:
            c0 += 1
    encoded_block.append((0, 0))

    encoded_block = np.array(encoded_block, dtype = int)
    return encoded_block

encoded_Y = list()
encoded_Cr = list()
encoded_Cb = list()
for i in range(len_y):
    encoded_Y.append(encode(Y_blocks[i]))
for i in range(len_c):
    encoded_Cb.append(encode(Cb_blocks[i]))
    encoded_Cr.append(encode(Cr_blocks[i]))
encoded_Y = np.array(encoded_Y, dtype = object)
encoded_Cr = np.array(encoded_Cr, dtype = object)
encoded_Cb = np.array(encoded_Cb, dtype = object)
print(encoded_Y[0], encoded_Cr[0], encoded_Cb[0], end = '\n')
print(encoded_Y[1], encoded_Cr[1], encoded_Cb[1], end = '\n')

# print(encoded_Y, encoded_Cb, encoded_Cr, sep = '\n')

# Huffman Tables
def get_cat(num): ##extracting position of msb to determine what length of bits it will need to be encoded
    num = int(abs(num))
    ans = 0
    pwr = 1
    while pwr < num:
        pwr <<= 1
        ans += 1
    return ans
##***********************************************************DANGER**********************


def make_freq_table(dc_freq, ac_freq, encoded_blocks):
    for block in encoded_blocks:
        #block of length 64 usually (we take 8*8 grids), first element is dc coeff, the rest are ac coeffs
        #dc block[0] is encoded as (0, value)
        dc = block[0]
        dc_run_length, dc_val = dc #dc_run_length will be 0
        
        cat = get_cat(dc_val)
        dc_freq[dc_run_length, cat] += 1
        
        for i in range(1, len(block)):
            
            #ac block[i] is encoded as (run length of zeros before value max 15, value of ith non zero ac coefficient)
            ac_i = block[i]
            run_length, value = ac_i
            cat = get_cat(value) # find the category of value
            ac_freq[run_length, cat] += 1
            
dc_huffman_Y = np.zeros((1, 16), dtype=int)
ac_huffman_Y = np.zeros((16,16), dtype=int)
dc_huffman_C = np.zeros((1, 16), dtype=int)
ac_huffman_C = np.zeros((16,16), dtype=int)

make_freq_table(dc_huffman_Y, ac_huffman_Y, encoded_Y)
make_freq_table(dc_huffman_C, ac_huffman_C, encoded_Cr)
make_freq_table(dc_huffman_C, ac_huffman_C, encoded_Cb)



# make huffman codes
depth = 0
hufs = [[0x00, 0x00, 0x01, 0x05, 0x01, 0x01, 0x01, 0x01, 0x01, 0x01, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x01, 0x02, 0x03, 0x04, 0x05, 0x06, 0x07, 0x08, 0x09, 0x0a, 0x0b], 
        [0x10, 0x00, 0x02, 0x01, 0x03, 0x03, 0x02, 0x04, 0x03, 0x05, 0x05, 0x04, 0x04, 0x00, 0x00, 0x01, 0x7d, 0x01, 0x02, 0x03, 0x00, 0x04, 0x11, 0x05, 0x12, 0x21, 0x31, 0x41, 0x06, 0x13, 0x51, 0x61, 0x07, 0x22, 0x71, 0x14, 0x32, 0x81, 0x91, 0xa1, 0x08, 0x23, 0x42, 0xb1, 0xc1, 0x15, 0x52, 0xd1, 0xf0, 0x24, 0x33, 0x62, 0x72, 0x82, 0x09, 0x0a, 0x16, 0x17, 0x18, 0x19, 0x1a
, 0x25, 0x26, 0x27, 0x28, 0x29, 0x2a, 0x34, 0x35, 0x36, 0x37, 0x38
, 0x39, 0x3a, 0x43, 0x44, 0x45, 0x46, 0x47, 0x48, 0x49, 0x4a, 0x53
, 0x54, 0x55, 0x56, 0x57, 0x58, 0x59, 0x5a, 0x63, 0x64, 0x65, 0x66
, 0x67, 0x68, 0x69, 0x6a, 0x73, 0x74, 0x75, 0x76, 0x77, 0x78, 0x79
, 0x7a, 0x83, 0x84, 0x85, 0x86, 0x87, 0x88, 0x89, 0x8a, 0x92, 0x93
, 0x94, 0x95, 0x96, 0x97, 0x98, 0x99, 0x9a, 0xa2, 0xa3, 0xa4, 0xa5
, 0xa6, 0xa7, 0xa8, 0xa9, 0xaa, 0xb2, 0xb3, 0xb4, 0xb5, 0xb6, 0xb7
, 0xb8, 0xb9, 0xba, 0xc2, 0xc3, 0xc4, 0xc5, 0xc6, 0xc7, 0xc8, 0xc9
, 0xca, 0xd2, 0xd3, 0xd4, 0xd5, 0xd6, 0xd7, 0xd8, 0xd9, 0xda, 0xe1
, 0xe2, 0xe3, 0xe4, 0xe5, 0xe6, 0xe7, 0xe8, 0xe9, 0xea, 0xf1, 0xf2
, 0xf3, 0xf4, 0xf5, 0xf6, 0xf7, 0xf8, 0xf9, 0xfa],
       []]
def get_codebook(freq):
    codebook = np.empty(freq.shape, dtype = object)
    def get_code_lengths(freq):

        def make_tree(freq):
            global code
            q = PriorityQueue()
            m = len(freq)
            n = len(freq[0])

            cnt = 0
            for i in range(m):
                for j in range(n):
                    if freq[i,j]:
                        q.put((freq[i,j], str((i,j))))
                        cnt += 1

            for i in range(cnt-1):
                a = q.get()
                b = q.get()
                q.put((a[0]+b[0], str("[" + a[1] + "," + b[1] + "]")))

            tree = eval(q.get()[1])
            return tree

        tree = make_tree(freq)
        code_lengths = np.zeros(freq.shape, dtype=int)

        def get_depths(arr, code_lengths):
            global depth
            if type(arr[1]) == int and type(arr[0]) == int:
                code_lengths[arr[0], arr[1]] = depth
            else:
                depth += 1
                get_depths(arr[0], code_lengths)
                get_depths(arr[1], code_lengths)
                depth-=1

        get_depths(tree, code_lengths)
        return code_lengths
    
    code_lengths = get_code_lengths(freq)
    length_symbol_pairs = []
    
    m, n = freq.shape
    for i in range(m):
        for j in range(n):
            if code_lengths[i, j]:
                length_symbol_pairs.append((code_lengths[i, j], (i, j)))
    length_symbol_pairs.sort()
    
    def get_code(code, length):
        st = bin(code).replace("0b", "")[::-1]
        while len(st) < length:
            st += '0'
        return st[::-1]
    
    code = 0
    cur_len = 0
    for length, symbol in length_symbol_pairs:
        while cur_len < length:  #if there is no symbol with the current length
            cur_len += 1
            code *= 2
        codebook[symbol[0], symbol[1]] = get_code(code, length)
        code += 1
    
    return codebook, length_symbol_pairs

codes_dc_Y, sorted_symbols_dc_Y = get_codebook(dc_huffman_Y)
codes_ac_Y, sorted_symbols_ac_Y = get_codebook(ac_huffman_Y)
codes_dc_C, sorted_symbols_dc_C = get_codebook(dc_huffman_C)
codes_ac_C, sorted_symbols_ac_C = get_codebook(ac_huffman_C)

def to_two_bytes(num):
    bh = bin(num).replace("0b", "")
    bh = bh[::-1]
    while len(bh) < 16:
        bh += '0'
    bh = bh[::-1]

    return [int(bh[:8], 2), int(bh[8:], 2)]

# Writing jpeg image in binary
jpeg_image = [0xff, 0xd8, 0xff, 0xe0, 0x00, 0x10]

header = [0x4a, 0x46, 0x49, 0x46, 0x00, 0x01, 0x01, 0x01, 0x00, 0x48, 0x00, 0x48, 0x00, 0x00]
jpeg_image.extend(header)

luminance_quantisation_table = [0xff, 0xdb, 0x00, 0x43, 0x00]
luminance_quantisation_table.extend([int(ele) for ele in spiral_traversal(Y_q)])
jpeg_image.extend(luminance_quantisation_table)

chrominance_quantisation_table = [0xff, 0xdb, 0x00, 0x43, 0x01]
chrominance_quantisation_table.extend([int(ele) for ele in spiral_traversal(C_q)])
jpeg_image.extend(chrominance_quantisation_table)

start_of_frame = [0xff, 0xc0, 0x00, 0x11, 0x08]
start_of_frame.extend(to_two_bytes(old_shape[0]))
start_of_frame.extend(to_two_bytes(old_shape[1]))
start_of_frame.extend([0x03, 0x01, 0x22, 0x00, 0x02, 0x11, 0x01, 0x03, 0x11, 0x01])
jpeg_image.extend(start_of_frame)

def parse_table(sorted_symbols, typ):
    table = [0xff, 0xc4]
    ln = 19
    symbols = []
    freqs = [0 for i in range(16)]
    
    for length, symbol in sorted_symbols:
        freqs[length] += 1
        symbols.append(symbol[0] * 16 + symbol[1])
        ln += 1
    
    table.extend(to_two_bytes(ln))
    table.append(typ)
    table.extend(freqs)
    table.extend(symbols)
    return table

jpeg_image.extend(parse_table(sorted_symbols_dc_Y, 0x00))
jpeg_image.extend(parse_table(sorted_symbols_ac_Y, 0x10))
jpeg_image.extend(parse_table(sorted_symbols_dc_C, 0x01))
jpeg_image.extend(parse_table(sorted_symbols_ac_C, 0x11))

start_of_scan = [0xff, 0xda, 0x00, 0x0c, 0x03, 0x01, 0x00, 0x02, 0x11, 0x03, 0x11, 0x00, 0x3f, 0x00]
jpeg_image.extend(start_of_scan)

def bit_rep(value):
    if value == 0:
        return ''
    else : 
        v = bin(abs(value)).replace("0b", '')
        if value>0 :
            return v
        else:
            return v.replace('0', 'a').replace('1','0').replace('a','1')

def encode_data(encoded_Y, encoded_Cb, encoded_Cr, codes_dc_Y, code_dc_C, codes_ac_Y, codes_ac_C):
    s = ''
    def encode_block(block, dc_codebook, ac_codebook):
        ans = ''
        dc_val = block[0][1]
        cat = get_cat(dc_val)
        ans+=(dc_codebook[0,cat]+bit_rep(dc_val))
        
        for i in range(1, len(block)):
            run, val = block[i]
            ans+=(ac_codebook[run, get_cat(val)]+bit_rep(val))
        print(ans)
        return ans
    
    for i in range(len(encoded_Y)):
        s += encode_block(encoded_Y[i], codes_dc_Y, codes_ac_Y)
    for i in range(len(encoded_Cb)):
        s += encode_block(encoded_Cb[i], codes_dc_C, codes_ac_C)
    for i in range(len(encoded_Cr)):
        s += encode_block(encoded_Cr[i], codes_dc_C, codes_ac_C)
        
    #pad with 0s
    a = len(s)%8
    s+='0'*a
    
    data = [int(s[i:i+8], 2) for i in range(0, len(s), 8)]
    print(data)
    fin_data = []
    for ele in data:
        fin_data.append(ele)
        if ele == 0xff:
            fin_data.append(0x00)
    return fin_data
    
jpeg_image.extend(encode_data(encoded_Y, encoded_Cb, encoded_Cr, codes_dc_Y, codes_dc_C, codes_ac_Y, codes_ac_C))

end_of_image = [0xff, 0xd9]
jpeg_image.extend(end_of_image)

try:
    with open("my_image.jpeg", 'wb') as f:
        for byte in jpeg_image:
            f.write(byte.to_bytes(1, byteorder='big'))
except Exception as e:
    print(e)

file = open('my_image.jpeg', 'rb')
data = file.read()
s = data.hex()
print(s)

l = re.findall('..?', s)
d = np.reshape(l, (-1,4))
d = [' '.join(ele) for ele in d]
for ele in d:
    pass
#     print(ele)
file.close()

[[  0 338]
 [ 15   0]
 [ 15   0]
 [ 15   0]
 [  0   0]] [[ 0  0]
 [15  0]
 [15  0]
 [15  0]
 [ 0  0]] [[   0 -680]
 [  15    0]
 [  15    0]
 [  15    0]
 [   0    0]]
[[ 0  0]
 [15  0]
 [15  0]
 [15  0]
 [ 0  0]] [[ 0  0]
 [15  0]
 [15  0]
 [15  0]
 [ 0  0]] [[ 0  0]
 [15  0]
 [15  0]
 [15  0]
 [ 0  0]]
111111110101001010010010000
010010010000
010010010000
010010010000
010010010000
010010010000
010010010000
010010010000
010010010000
010010010000
010010010000
010010010000
010010010000
010010010000
010010010000
010010010000
010010010000
010010010000
010010010000
11000110101111011100100111011001111000110101100000010101011100110101000101000110100111000110101111101101000011110101001001010011101111010000111110000000001111000111111000000111100000000000011111100011010000000111100000000000
100001001010111111001001110111001011101111010011001011000111000111001010011110001101000011101101011111011101011111110100101111000111110000001101001100110101101111010111100000011010000110100100100011010111100

ValueError: cannot reshape array of size 15762 into shape (4)

In [128]:
# BMP to JPEG conversion
img = cv2.imread('snail.bmp')

s = img.shape
new_shape = (((s[0] + 15) // 16) * 16, ((s[1] + 15) // 16) * 16, s[2])
new_image = np.zeros(new_shape, dtype = 'float32')
new_image[:s[0], :s[1], :] = img
img = np.array(new_image)
old_shape = s
s = new_shape
s1 = (s[0] // 2, s[1] // 2)

# todo - padding

# conversion from RGB format to YCbCr (Y -> Luminance, Cb -> blue chrominance, Cr -> red chrominance)
imgYCC = cv2.cvtColor(img, cv2.COLOR_BGR2YCR_CB)
Y = imgYCC[:,:,0]
Cb = np.zeros(s1, dtype = int)
Cr = np.zeros(s1, dtype = int)

# average every 2x2 block for Cb and Cr
for i in range(s[0]//2):
    for j in range(s[1]//2):
        x = i*2
        y = j*2
        avg = (imgYCC[x, y, 1] + imgYCC[x, y + 1, 1] + imgYCC[x + 1, y, 1] + imgYCC[x + 1, y + 1, 1]) // 4
        Cr[i, j] = avg
        avg = (imgYCC[x][y][2] + imgYCC[x][y + 1][2] + imgYCC[x + 1][y][2] + imgYCC[x + 1][y + 1][2]) // 4
        Cb[i, j] = avg

# Discrete Cosine Transform
Y -= 128
Cb -= 128
Cr -= 128
Y_blocks = []
Cb_blocks = []
Cr_blocks = []
for i in range(0, s1[0], 8):
    for j in range(0, s1[1], 8):
        grid = np.float32(Cb[i:i+8, j:j+8])
        Cb_blocks.append(cv2.dct(grid))
        grid = np.float32(Cr[i:i+8, j:j+8])
        Cr_blocks.append(cv2.dct(grid))
        grid = np.float32(Y[i*2:i*2+8, j*2:j*2+8])
        Y_blocks.append(cv2.dct(grid))
        grid = np.float32(Y[i*2:i*2+8, j*2+8:j*2+16])
        Y_blocks.append(cv2.dct(grid))
        grid = np.float32(Y[i*2+8:i*2+16, j*2:j*2+8])
        Y_blocks.append(cv2.dct(grid))
        grid = np.float32(Y[i*2+8:i*2+16, j*2+8:j*2+16])
        Y_blocks.append(cv2.dct(grid))

len_y = len(Y_blocks)
len_c = len(Cb_blocks)

# Quantization

def spiralo(block):
    p = 8
    arr = []
    k = 0
    for c in range(p):
        for i in range(0, k+1):
            if k%2!=0 :
                arr.append(block[i][k-i])
            else :
                arr.append(block[k-i][i])
        k+=1

    k = 1
    for c in range(p-1):
        j = p
        for i in range(k, p):
            j-=1
            if k%2 == 0:
                arr.append(block[i][j])
            else:
                arr.append(block[j][i])
        k+=1
    return arr

# Quantization
myArr = [
[0x03, 0x02, 0x02, 0x03, 0x02, 0x02, 0x03, 0x03],
[0x03, 0x03, 0x04, 0x03, 0x03, 0x04, 0x05, 0x08],
[0x05, 0x05, 0x04, 0x04, 0x05, 0x0a, 0x07, 0x07],
[0x06, 0x08, 0x0c, 0x0a, 0x0c, 0x0c, 0x0b, 0x0a],
[0x0b, 0x0b, 0x0d, 0x0e, 0x12, 0x10, 0x0d, 0x0e],
[0x11, 0x0e, 0x0b, 0x0b, 0x10, 0x16, 0x10, 0x11],
[0x13, 0x14, 0x15, 0x15, 0x15, 0x0c, 0x0f, 0x17],
[0x18, 0x16, 0x14, 0x18, 0x12, 0x14, 0x15, 0x14]
]
myArr2 = [
[0x03, 0x04, 0x04, 0x05, 0x04, 0x05, 0x09, 0x05],
[0x05, 0x09, 0x14, 0x0d, 0x0b, 0x0d, 0x14, 0x14],
[0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14],
[0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14],
[0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14],
[0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14],
[0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14],
[0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14]
]
spiral = [[(i, j) for j in range(8)] for i in range(8)]
spiral = spiralo(spiral)
Y_q = np.array([[4., 3, 4, 4, 4, 6, 11, 15], [3, 3, 3, 4, 5, 8, 14, 19], [3, 4, 4, 5, 8, 12, 16, 20], [4, 5, 6, 7, 12, 14, 18, 20], [6, 6, 9, 11, 14, 17, 21, 23], [9, 12, 12, 18, 23, 22, 25, 21], [11, 13, 15, 17, 21, 23, 25, 21], [13, 12, 12, 13, 16, 19, 21, 21]])
C_q = np.array([[4., 4, 6, 10, 21, 21, 21, 21], [4, 5, 6, 21, 21, 21, 21, 21], [6, 6, 12, 21, 21, 21, 21, 21], [10, 14, 21, 21, 21, 21, 21, 21], [21, 21, 21, 21, 21, 21, 21, 21], [21, 21, 21, 21, 21, 21, 21, 21], [21, 21, 21, 21, 21, 21, 21, 21], [21, 21, 21, 21, 21, 21, 21, 21]])
cur = 0
for i in range(8):
    for j in range(8):
        Y_q[spiral[cur][0]][spiral[cur][1]] = myArr[i][j]
        C_q[spiral[cur][0]][spiral[cur][1]] = myArr2[i][j]
        cur += 1
for i in range(len_y):
    Y_blocks[i] //= Y_q
for i in range(len_c):
    Cb_blocks[i] //= C_q
for i in range(len_c):
    Cr_blocks[i] //= C_q

# Run length encoding
def spiral_traversal(block):
    p = 8
    arr = np.array([])
    k = 0
    for c in range(p):
        for i in range(0, k+1):
            if k%2!=0 :
                arr = np.append(arr, block[i][k-i])
            else :
                arr = np.append(arr, block[k-i][i])
        k+=1

    k = 1
    for c in range(p-1):
        j = p
        for i in range(k, p):
            j-=1
            if k%2 == 0:
                arr = np.append(arr, block[i][j])
            else:
                arr = np.append(arr, block[j][i])
        k+=1
    return arr

dc = 0
def encode(block):
    global dc
    temp = spiral_traversal(block)
    l = len(temp)

    encoded_block = []
    encoded_block.append((0, temp[0] - dc))
    dc = temp[0]
    c0 = 0
    for i in range(1, l):
        if temp[i] or c0==15 :
            encoded_block.append((c0, temp[i]))
            c0 = 0
        else:
            c0 += 1
    encoded_block.append((0, 0))

    encoded_block = np.array(encoded_block, dtype = int)
    return encoded_block

encoded_Y = list()
encoded_Cr = list()
encoded_Cb = list()
for i in range(len_y):
    encoded_Y.append(encode(Y_blocks[i]))
for i in range(len_c):
    encoded_Cb.append(encode(Cb_blocks[i]))
    encoded_Cr.append(encode(Cr_blocks[i]))
encoded_Y = np.array(encoded_Y, dtype = object)
encoded_Cr = np.array(encoded_Cr, dtype = object)
encoded_Cb = np.array(encoded_Cb, dtype = object)

# print(encoded_Y, encoded_Cb, encoded_Cr, sep = '\n')

# Huffman Tables
def get_cat(num): ##extracting position of msb to determine what length of bits it will need to be encoded
    num = int(abs(num))
    ans = 0
    pwr = 1
    while pwr < num:
        pwr <<= 1
        ans += 1
    return ans
##***********************************************************DANGER**********************


def make_freq_table(dc_freq, ac_freq, encoded_blocks):
    for block in encoded_blocks:
        #block of length 64 usually (we take 8*8 grids), first element is dc coeff, the rest are ac coeffs
        #dc block[0] is encoded as (0, value)
        dc = block[0]
        dc_run_length, dc_val = dc #dc_run_length will be 0
        
        cat = get_cat(dc_val)
        dc_freq[dc_run_length, cat] += 1
        
        for i in range(1, len(block)):
            
            #ac block[i] is encoded as (run length of zeros before value max 15, value of ith non zero ac coefficient)
            ac_i = block[i]
            run_length, value = ac_i
            cat = get_cat(value) # find the category of value
            ac_freq[run_length, cat] += 1
            
dc_huffman_Y = np.zeros((1, 16), dtype=int)
ac_huffman_Y = np.zeros((16,16), dtype=int)
dc_huffman_C = np.zeros((1, 16), dtype=int)
ac_huffman_C = np.zeros((16,16), dtype=int)

make_freq_table(dc_huffman_Y, ac_huffman_Y, encoded_Y)
make_freq_table(dc_huffman_C, ac_huffman_C, encoded_Cr)
make_freq_table(dc_huffman_C, ac_huffman_C, encoded_Cb)



# make huffman codes
depth = 0
hufs = [[0x00, 0x01, 0x05, 0x01, 0x01, 0x01, 0x01, 0x01, 0x01, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x01, 0x02, 0x03, 0x04, 0x05, 0x06, 0x07, 0x08, 0x09, 0x0a, 0x0b], 
        [0x00, 0x02, 0x01, 0x03, 0x03, 0x02, 0x04, 0x03, 0x05, 0x05, 0x04, 0x04, 0x00, 0x00, 0x01, 0x7d, 0x01, 0x02, 0x03, 0x00, 0x04, 0x11, 0x05, 0x12, 0x21, 0x31, 0x41, 0x06, 0x13, 0x51, 0x61, 0x07, 0x22, 0x71, 0x14, 0x32, 0x81, 0x91, 0xa1, 0x08, 0x23, 0x42, 0xb1, 0xc1, 0x15, 0x52, 0xd1, 0xf0, 0x24, 0x33, 0x62, 0x72, 0x82, 0x09, 0x0a, 0x16, 0x17, 0x18, 0x19, 0x1a
, 0x25, 0x26, 0x27, 0x28, 0x29, 0x2a, 0x34, 0x35, 0x36, 0x37, 0x38
, 0x39, 0x3a, 0x43, 0x44, 0x45, 0x46, 0x47, 0x48, 0x49, 0x4a, 0x53
, 0x54, 0x55, 0x56, 0x57, 0x58, 0x59, 0x5a, 0x63, 0x64, 0x65, 0x66
, 0x67, 0x68, 0x69, 0x6a, 0x73, 0x74, 0x75, 0x76, 0x77, 0x78, 0x79
, 0x7a, 0x83, 0x84, 0x85, 0x86, 0x87, 0x88, 0x89, 0x8a, 0x92, 0x93
, 0x94, 0x95, 0x96, 0x97, 0x98, 0x99, 0x9a, 0xa2, 0xa3, 0xa4, 0xa5
, 0xa6, 0xa7, 0xa8, 0xa9, 0xaa, 0xb2, 0xb3, 0xb4, 0xb5, 0xb6, 0xb7
, 0xb8, 0xb9, 0xba, 0xc2, 0xc3, 0xc4, 0xc5, 0xc6, 0xc7, 0xc8, 0xc9
, 0xca, 0xd2, 0xd3, 0xd4, 0xd5, 0xd6, 0xd7, 0xd8, 0xd9, 0xda, 0xe1
, 0xe2, 0xe3, 0xe4, 0xe5, 0xe6, 0xe7, 0xe8, 0xe9, 0xea, 0xf1, 0xf2
, 0xf3, 0xf4, 0xf5, 0xf6, 0xf7, 0xf8, 0xf9, 0xfa],
       []]



def get_codebook(freq):
    codebook = np.empty(freq.shape, dtype = object)
    def code_lengths_from_huf(huf_table, m, n):
        #for dc m = 1, n = 16
        code_lengths = np.zeros((m,n), dtype=int)
        counts = np.zeros((1,17), dtype=int)
        for length in range(1, 17):
            counts[length] = huf_table[length-1]
        for i in range(m):
            for j in range(n):
                
        return code_lengths
    
    def get_code_lengths(freq):

        def make_tree(freq):
            global code
            q = PriorityQueue()
            m = len(freq)
            n = len(freq[0])

            cnt = 0
            for i in range(m):
                for j in range(n):
                    if freq[i,j]:
                        q.put((freq[i,j], str((i,j))))
                        cnt += 1

            for i in range(cnt-1):
                a = q.get()
                b = q.get()
                q.put((a[0]+b[0], str("[" + a[1] + "," + b[1] + "]")))

            tree = eval(q.get()[1])
            return tree

        tree = make_tree(freq)
        code_lengths = np.zeros(freq.shape, dtype=int)

        depth = 0
        def get_depths(arr, code_lengths):
            global depth
            if type(arr[1]) == int and type(arr[0]) == int:
                code_lengths[arr[0], arr[1]] = min(depth, 16)
            else:
                depth += 1
                get_depths(arr[0], code_lengths)
                get_depths(arr[1], code_lengths)
                depth-=1

        get_depths(tree, code_lengths)
        return code_lengths
    
    code_lengths = get_code_lengths(freq)
    
    length_symbol_pairs = []
    
    m, n = freq.shape
    for i in range(m):
        for j in range(n):
            if code_lengths[i, j]:
                length_symbol_pairs.append((code_lengths[i, j], (i, j)))
    length_symbol_pairs.sort()
    
    def get_code(code, length):
        st = bin(code).replace("0b", "")[::-1]
        while len(st) < length:
            st += '0'
        return st[::-1]
    
    code = 0
    cur_len = 0
    last = length_symbol_pairs[-1]
    length_symbol_pairs[-1] = (last[0] + 1, last[1])
    for length, symbol in length_symbol_pairs:
        while cur_len < length:  #if there is no symbol with the current length
            cur_len += 1
            code *= 2
        codebook[symbol[0], symbol[1]] = get_code(code, length)
        code += 1
    
    return codebook, length_symbol_pairs

codes_dc_Y, sorted_symbols_dc_Y = get_codebook(dc_huffman_Y)
codes_ac_Y, sorted_symbols_ac_Y = get_codebook(ac_huffman_Y)
codes_dc_C, sorted_symbols_dc_C = get_codebook(dc_huffman_C)
codes_ac_C, sorted_symbols_ac_C = get_codebook(ac_huffman_C)
print(list(codes_dc_Y))

def to_two_bytes(num):
    bh = bin(num).replace("0b", "")
    bh = bh[::-1]
    while len(bh) < 16:
        bh += '0'
    bh = bh[::-1]

    return [int(bh[:8], 2), int(bh[8:], 2)]

# Writing jpeg image in binary
jpeg_image = [0xff, 0xd8, 0xff, 0xe0, 0x00, 0x10]

header = [0x4a, 0x46, 0x49, 0x46, 0x00, 0x01, 0x01, 0x01, 0x00, 0x48, 0x00, 0x48, 0x00, 0x00]
jpeg_image.extend(header)

luminance_quantisation_table = [0xff, 0xdb, 0x00, 0x43, 0x00]
luminance_quantisation_table.extend([int(ele) for ele in spiral_traversal(Y_q)])
jpeg_image.extend(luminance_quantisation_table)

chrominance_quantisation_table = [0xff, 0xdb, 0x00, 0x43, 0x01]
chrominance_quantisation_table.extend([int(ele) for ele in spiral_traversal(C_q)])
jpeg_image.extend(chrominance_quantisation_table)

start_of_frame = [0xff, 0xc0, 0x00, 0x11, 0x08]
start_of_frame.extend(to_two_bytes(old_shape[0]))
start_of_frame.extend(to_two_bytes(old_shape[1]))
start_of_frame.extend([0x03, 0x01, 0x22, 0x00, 0x02, 0x11, 0x01, 0x03, 0x11, 0x01])
jpeg_image.extend(start_of_frame)

def parse_table(sorted_symbols, typ):
    table = [0xff, 0xc4]
    ln = 19
    symbols = []
    freqs = [0 for i in range(16)]
    
    for length, symbol in sorted_symbols:
        print(length)
        freqs[length-1] += 1
        symbols.append(symbol[0] * 16 + symbol[1])
        ln += 1
    print()
    
    table.extend(to_two_bytes(ln))
    table.append(typ)
    table.extend(freqs)
    table.extend(symbols)
    return table

jpeg_image.extend(parse_table(sorted_symbols_dc_Y, 0x00))
jpeg_image.extend(parse_table(sorted_symbols_ac_Y, 0x10))
jpeg_image.extend(parse_table(sorted_symbols_dc_C, 0x01))
jpeg_image.extend(parse_table(sorted_symbols_ac_C, 0x11))

start_of_scan = [0xff, 0xda, 0x00, 0x0c, 0x03, 0x01, 0x00, 0x02, 0x11, 0x03, 0x11, 0x00, 0x3f, 0x00]
jpeg_image.extend(start_of_scan)

def bit_rep(value):
    if value == 0:
        return ''
    else : 
        v = bin(abs(value)).replace("0b", '')
        if value>0 :
            return v
        else:
            return v.replace('0', 'a').replace('1','0').replace('a','1')

def encode_data(encoded_Y, encoded_Cb, encoded_Cr, codes_dc_Y, code_dc_C, codes_ac_Y, codes_ac_C):
    s = ''
    def encode_block(block, dc_codebook, ac_codebook):
        ans = ''
        dc_val = block[0][1]
        cat = get_cat(dc_val)
        ans+=(dc_codebook[0,cat]+bit_rep(dc_val))
        
        for i in range(1, len(block)):
            run, val = block[i]
            ans+=(ac_codebook[run, get_cat(val)]+bit_rep(val))
        return ans
    
    for i in range(len(encoded_Y)):
        s += encode_block(encoded_Y[i], codes_dc_Y, codes_ac_Y)
        if (i % 4 == 3):
            s += encode_block(encoded_Cb[i // 4], codes_dc_C, codes_ac_C)
            s += encode_block(encoded_Cr[i // 4], codes_dc_C, codes_ac_C)
        
    #pad with 0s
    a = len(s)%8
    s+='0'*a
    
    data = [int(s[i:i+8], 2) for i in range(0, len(s), 8)]
    fin_data = []
    for ele in data:
        fin_data.append(ele)
        if ele == 0xff:
            fin_data.append(0x00)
    return fin_data
    
jpeg_image.extend(encode_data(encoded_Y, encoded_Cb, encoded_Cr, codes_dc_Y, codes_dc_C, codes_ac_Y, codes_ac_C))

end_of_image = [0xff, 0xd9]
jpeg_image.extend(end_of_image)

try:
    with open("my_image.jpeg", 'wb') as f:
        for byte in jpeg_image:
            f.write(byte.to_bytes(1, byteorder='big'))
except Exception as e:
    print(e)

file = open('my_image.jpeg', 'rb')
data = file.read()
s = data.hex()

l = re.findall('..?', s)
d = np.reshape(l, (-1,6))
d = [' '.join(ele) for ele in d]
print(len(d))
for ele in d:
    print(ele)

file.close()

[[2 3 3 3 3 3 4 5 6 7 8 9 0 0 0 0]]
[[2 3 3 3 3 3 4 5 6 7 8 9 0 0 0 0]]
[[2 3 3 3 3 3 4 5 6 7 8 9 0 0 0 0]]
[[2 3 3 3 3 3 4 5 6 7 8 9 0 0 0 0]]
[array(['0', '1111110', '11111110', '11110', '1100', '100', '101', '1101',
       '1110', '111110', None, None, None, None, None, None], dtype=object)]
1
3
3
4
4
4
5
6
7
8

2
3
3
3
4
4
4
4
5
6
6
6
7
7
7
8
8
8
9
9
10
10
10
10
11
11
11
11
12
12
12
12
13
13
13
13
13
14
14
14
14
14
15

1
2
4
4
5
5
5
7
7
7
8

1
3
3
4
4
5
6
6
6
7
7
8
8
8
8
8
9
9
10
10
10
10
10
11
11
11
11
12
13
13
13
13
13
14

2631
ff d8 ff e0 00 10
4a 46 49 46 00 01
01 01 00 48 00 48
00 00 ff db 00 43
00 03 02 02 03 02
02 03 03 03 03 04
03 03 04 05 08 05
05 04 04 05 0a 07
07 06 08 0c 0a 0c
0c 0b 0a 0b 0b 0d
0e 12 10 0d 0e 11
0e 0b 0b 10 16 10
11 13 14 15 15 15
0c 0f 17 18 16 14
18 12 14 15 14 ff
db 00 43 01 03 04
04 05 04 05 09 05
05 09 14 0d 0b 0d
14 14 14 14 14 14
14 14 14 14 14 14
14 14 14 14 14 14
14 14 14 14 14 14
14 14 14 14 14 14
14 14 14 14 14 14
14 14 14 14 14 14
14 14 14 1

In [153]:
# BMP to JPEG conversion
img = cv2.imread('snail.bmp')

s = img.shape
new_shape = (((s[0] + 15) // 16) * 16, ((s[1] + 15) // 16) * 16, s[2])
new_image = np.zeros(new_shape, dtype = 'float32')
new_image[:s[0], :s[1], :] = img
img = np.array(new_image)
old_shape = s
s = new_shape
s1 = (s[0] // 2, s[1] // 2)

# todo - padding

# conversion from RGB format to YCbCr (Y -> Luminance, Cb -> blue chrominance, Cr -> red chrominance)
imgYCC = cv2.cvtColor(img, cv2.COLOR_BGR2YCR_CB)
Y = imgYCC[:,:,0]
Cb = np.zeros(s1, dtype = int)
Cr = np.zeros(s1, dtype = int)

# average every 2x2 block for Cb and Cr
for i in range(s[0]//2):
    for j in range(s[1]//2):
        x = i*2
        y = j*2
        avg = (imgYCC[x, y, 1] + imgYCC[x, y + 1, 1] + imgYCC[x + 1, y, 1] + imgYCC[x + 1, y + 1, 1]) // 4
        Cb[i, j] = avg
        avg = (imgYCC[x][y][2] + imgYCC[x][y + 1][2] + imgYCC[x + 1][y][2] + imgYCC[x + 1][y + 1][2]) // 4
        Cr[i, j] = avg

# Discrete Cosine Transform
Y -= 128
Cb -= 128
Cr -= 128
Y_blocks = []
Cb_blocks = []
Cr_blocks = []
for i in range(0, s1[0], 8):
    for j in range(0, s1[1], 8):
        grid = np.float32(Cb[i:i+8, j:j+8])
        Cb_blocks.append(cv2.dct(grid))
        grid = np.float32(Cr[i:i+8, j:j+8])
        Cr_blocks.append(cv2.dct(grid))
        grid = np.float32(Y[i*2:i*2+8, j*2:j*2+8])
        Y_blocks.append(cv2.dct(grid))
        grid = np.float32(Y[i*2:i*2+8, j*2+8:j*2+16])
        Y_blocks.append(cv2.dct(grid))
        grid = np.float32(Y[i*2+8:i*2+16, j*2:j*2+8])
        Y_blocks.append(cv2.dct(grid))
        grid = np.float32(Y[i*2+8:i*2+16, j*2+8:j*2+16])
        Y_blocks.append(cv2.dct(grid))

len_y = len(Y_blocks)
len_c = len(Cb_blocks)

# Quantization

def spiralo(block):
    p = 8
    arr = []
    k = 0
    for c in range(p):
        for i in range(0, k+1):
            if k%2!=0 :
                arr.append(block[i][k-i])
            else :
                arr.append(block[k-i][i])
        k+=1

    k = 1
    for c in range(p-1):
        j = p
        for i in range(k, p):
            j-=1
            if k%2 == 0:
                arr.append(block[i][j])
            else:
                arr.append(block[j][i])
        k+=1
    return arr

# Quantization
myArr = [
[0x03, 0x02, 0x02, 0x03, 0x02, 0x02, 0x03, 0x03],
[0x03, 0x03, 0x04, 0x03, 0x03, 0x04, 0x05, 0x08],
[0x05, 0x05, 0x04, 0x04, 0x05, 0x0a, 0x07, 0x07],
[0x06, 0x08, 0x0c, 0x0a, 0x0c, 0x0c, 0x0b, 0x0a],
[0x0b, 0x0b, 0x0d, 0x0e, 0x12, 0x10, 0x0d, 0x0e],
[0x11, 0x0e, 0x0b, 0x0b, 0x10, 0x16, 0x10, 0x11],
[0x13, 0x14, 0x15, 0x15, 0x15, 0x0c, 0x0f, 0x17],
[0x18, 0x16, 0x14, 0x18, 0x12, 0x14, 0x15, 0x14]
]
myArr2 = [
[0x03, 0x04, 0x04, 0x05, 0x04, 0x05, 0x09, 0x05],
[0x05, 0x09, 0x14, 0x0d, 0x0b, 0x0d, 0x14, 0x14],
[0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14],
[0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14],
[0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14],
[0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14],
[0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14],
[0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14, 0x14]
]
spiral = [[(i, j) for j in range(8)] for i in range(8)]
spiral = spiralo(spiral)
Y_q = np.array([[4., 3, 4, 4, 4, 6, 11, 15], [3, 3, 3, 4, 5, 8, 14, 19], [3, 4, 4, 5, 8, 12, 16, 20], [4, 5, 6, 7, 12, 14, 18, 20], [6, 6, 9, 11, 14, 17, 21, 23], [9, 12, 12, 18, 23, 22, 25, 21], [11, 13, 15, 17, 21, 23, 25, 21], [13, 12, 12, 13, 16, 19, 21, 21]])
C_q = np.array([[4., 4, 6, 10, 21, 21, 21, 21], [4, 5, 6, 21, 21, 21, 21, 21], [6, 6, 12, 21, 21, 21, 21, 21], [10, 14, 21, 21, 21, 21, 21, 21], [21, 21, 21, 21, 21, 21, 21, 21], [21, 21, 21, 21, 21, 21, 21, 21], [21, 21, 21, 21, 21, 21, 21, 21], [21, 21, 21, 21, 21, 21, 21, 21]])
cur = 0
for i in range(8):
    for j in range(8):
        Y_q[spiral[cur][0]][spiral[cur][1]] = myArr[i][j]
        C_q[spiral[cur][0]][spiral[cur][1]] = myArr2[i][j]
        cur += 1
for i in range(len_y):
    Y_blocks[i] = np.around(Y_blocks[i] / Y_q)
for i in range(len_c):
    Cb_blocks[i] = np.around(Cb_blocks[i] / C_q)
for i in range(len_c):
    Cr_blocks[i] = np.around(Cr_blocks[i] / C_q)

# Run length encoding
def spiral_traversal(block):
    p = 8
    arr = np.array([])
    k = 0
    for c in range(p):
        for i in range(0, k+1):
            if k%2!=0 :
                arr = np.append(arr, block[i][k-i])
            else :
                arr = np.append(arr, block[k-i][i])
        k+=1

    k = 1
    for c in range(p-1):
        j = p
        for i in range(k, p):
            j-=1
            if k%2 == 0:
                arr = np.append(arr, block[i][j])
            else:
                arr = np.append(arr, block[j][i])
        k+=1
    return arr

dc = 0
def encode(block):
    global dc
    temp = spiral_traversal(block)
    l = len(temp)

    encoded_block = []
    encoded_block.append((0, temp[0] - dc))
    dc = temp[0]
    c0 = 0
    cnz = 0
    for i in range(1, l):
        if temp[i]: cnz += 1
    for i in range(1, l):
        if cnz == 0:
            break
        if temp[i] or c0==15 :
            encoded_block.append((c0, temp[i]))
            c0 = 0
            if temp[i]:
                cnz -= 1
        else:
            c0 += 1
    encoded_block.append((0, 0))

    encoded_block = np.array(encoded_block, dtype = int)
    return encoded_block

encoded_Y = list()
encoded_Cr = list()
encoded_Cb = list()
for i in range(len_y):
    encoded_Y.append(encode(Y_blocks[i]))
for i in range(len_c):
    encoded_Cb.append(encode(Cb_blocks[i]))
    encoded_Cr.append(encode(Cr_blocks[i]))
encoded_Y = np.array(encoded_Y, dtype = object)
encoded_Cr = np.array(encoded_Cr, dtype = object)
encoded_Cb = np.array(encoded_Cb, dtype = object)

print(encoded_Y[:4], encoded_Cb[0], encoded_Cr[0], sep = '\n')

# print(encoded_Y, encoded_Cb, encoded_Cr, sep = '\n')

# Huffman Tables
def get_cat(num): ##extracting position of msb to determine what length of bits it will need to be encoded
    num = int(abs(num))
    ans = 0
    pwr = 1
    while pwr <= num:
        pwr <<= 1
        ans += 1
    return ans
##***********************************************************DANGER**********************


def make_freq_table(dc_freq, ac_freq, encoded_blocks):
    for block in encoded_blocks:
        #block of length 64 usually (we take 8*8 grids), first element is dc coeff, the rest are ac coeffs
        #dc block[0] is encoded as (0, value)
        dc = block[0]
        dc_run_length, dc_val = dc #dc_run_length will be 0
        
        cat = get_cat(dc_val)
        dc_freq[dc_run_length, cat] += 1
        
        for i in range(1, len(block)):
            
            #ac block[i] is encoded as (run length of zeros before value max 15, value of ith non zero ac coefficient)
            ac_i = block[i]
            run_length, value = ac_i
            cat = get_cat(value) # find the category of value
            ac_freq[run_length, cat] += 1
            
dc_huffman_Y = np.zeros((1, 16), dtype=int)
ac_huffman_Y = np.zeros((16,16), dtype=int)
dc_huffman_C = np.zeros((1, 16), dtype=int)
ac_huffman_C = np.zeros((16,16), dtype=int)

make_freq_table(dc_huffman_Y, ac_huffman_Y, encoded_Y)
make_freq_table(dc_huffman_C, ac_huffman_C, encoded_Cr)
make_freq_table(dc_huffman_C, ac_huffman_C, encoded_Cb)



# make huffman codes
depth = 0
hufs = [[0x00, 0x01, 0x05, 0x01, 0x01, 0x01, 0x01, 0x01, 0x01, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x01, 0x02, 0x03, 0x04, 0x05, 0x06, 0x07, 0x08, 0x09, 0x0a, 0x0b], 
        [0x00, 0x02, 0x01, 0x03, 0x03, 0x02, 0x04, 0x03, 0x05, 0x05, 0x04, 0x04, 0x00, 0x00, 0x01, 0x7d, 0x01, 0x02, 0x03, 0x00, 0x04, 0x11, 0x05, 0x12, 0x21, 0x31, 0x41, 0x06, 0x13, 0x51, 0x61, 0x07, 0x22, 0x71, 0x14, 0x32, 0x81, 0x91, 0xa1, 0x08, 0x23, 0x42, 0xb1, 0xc1, 0x15, 0x52, 0xd1, 0xf0, 0x24, 0x33, 0x62, 0x72, 0x82, 0x09, 0x0a, 0x16, 0x17, 0x18, 0x19, 0x1a
, 0x25, 0x26, 0x27, 0x28, 0x29, 0x2a, 0x34, 0x35, 0x36, 0x37, 0x38
, 0x39, 0x3a, 0x43, 0x44, 0x45, 0x46, 0x47, 0x48, 0x49, 0x4a, 0x53
, 0x54, 0x55, 0x56, 0x57, 0x58, 0x59, 0x5a, 0x63, 0x64, 0x65, 0x66
, 0x67, 0x68, 0x69, 0x6a, 0x73, 0x74, 0x75, 0x76, 0x77, 0x78, 0x79
, 0x7a, 0x83, 0x84, 0x85, 0x86, 0x87, 0x88, 0x89, 0x8a, 0x92, 0x93
, 0x94, 0x95, 0x96, 0x97, 0x98, 0x99, 0x9a, 0xa2, 0xa3, 0xa4, 0xa5
, 0xa6, 0xa7, 0xa8, 0xa9, 0xaa, 0xb2, 0xb3, 0xb4, 0xb5, 0xb6, 0xb7
, 0xb8, 0xb9, 0xba, 0xc2, 0xc3, 0xc4, 0xc5, 0xc6, 0xc7, 0xc8, 0xc9
, 0xca, 0xd2, 0xd3, 0xd4, 0xd5, 0xd6, 0xd7, 0xd8, 0xd9, 0xda, 0xe1
, 0xe2, 0xe3, 0xe4, 0xe5, 0xe6, 0xe7, 0xe8, 0xe9, 0xea, 0xf1, 0xf2
, 0xf3, 0xf4, 0xf5, 0xf6, 0xf7, 0xf8, 0xf9, 0xfa],
       [0x00, 0x03, 0x01, 0x01, 0x01, 0x01, 0x01, 0x01, 0x01
, 0x01, 0x01, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x01, 0x02, 0x03
, 0x04, 0x05, 0x06, 0x07, 0x08, 0x09, 0x0a, 0x0b],
       [0x00, 0x02, 0x01, 0x02, 0x04, 0x04, 0x03, 0x04, 0x07
, 0x05, 0x04, 0x04, 0x00, 0x01, 0x02, 0x77, 0x00, 0x01, 0x02, 0x03
, 0x11, 0x04, 0x05, 0x21, 0x31, 0x06, 0x12, 0x41, 0x51, 0x07, 0x61
, 0x71, 0x13, 0x22, 0x32, 0x81, 0x08, 0x14, 0x42, 0x91, 0xa1, 0xb1
, 0xc1, 0x09, 0x23, 0x33, 0x52, 0xf0, 0x15, 0x62, 0x72, 0xd1, 0x0a
, 0x16, 0x24, 0x34, 0xe1, 0x25, 0xf1, 0x17, 0x18, 0x19, 0x1a, 0x26
, 0x27, 0x28, 0x29, 0x2a, 0x35, 0x36, 0x37, 0x38, 0x39, 0x3a, 0x43
, 0x44, 0x45, 0x46, 0x47, 0x48, 0x49, 0x4a, 0x53, 0x54, 0x55, 0x56
, 0x57, 0x58, 0x59, 0x5a, 0x63, 0x64, 0x65, 0x66, 0x67, 0x68, 0x69
, 0x6a, 0x73, 0x74, 0x75, 0x76, 0x77, 0x78, 0x79, 0x7a, 0x82, 0x83
, 0x84, 0x85, 0x86, 0x87, 0x88, 0x89, 0x8a, 0x92, 0x93, 0x94, 0x95
, 0x96, 0x97, 0x98, 0x99, 0x9a, 0xa2, 0xa3, 0xa4, 0xa5, 0xa6, 0xa7
, 0xa8, 0xa9, 0xaa, 0xb2, 0xb3, 0xb4, 0xb5, 0xb6, 0xb7, 0xb8, 0xb9
, 0xba, 0xc2, 0xc3, 0xc4, 0xc5, 0xc6, 0xc7, 0xc8, 0xc9, 0xca, 0xd2
, 0xd3, 0xd4, 0xd5, 0xd6, 0xd7, 0xd8, 0xd9, 0xda, 0xe2, 0xe3, 0xe4
, 0xe5, 0xe6, 0xe7, 0xe8, 0xe9, 0xea, 0xf2, 0xf3, 0xf4, 0xf5, 0xf6
, 0xf7, 0xf8, 0xf9, 0xfa]]



def get_codebook(freq, index):
    codebook = np.empty(freq.shape, dtype = object)
    def get_code_lengths_from_huf(huf_table, m, n):
        #for dc m = 1, n = 16
        code_lengths = np.zeros((m,n), dtype=int)
        counts = np.zeros(17, dtype=int)
        for length in range(1, 17):
            counts[length] = huf_table[length-1]
        l = len(huf_table)
        ctr = 0
        for w in range(16, l):
            symbol = huf_table[w]
            i = symbol//16
            j = symbol%16
            while counts[ctr]==0: ctr+=1
#             print(ctr, i, j)
            code_lengths[i, j] = ctr
            counts[ctr]-=1
        print(code_lengths)
        return code_lengths
    
    def get_code_lengths(freq):

        def make_tree(freq):
            global code
            q = PriorityQueue()
            m = len(freq)
            n = len(freq[0])

            cnt = 0
            for i in range(m):
                for j in range(n):
                    if freq[i,j]:
                        q.put((freq[i,j], str((i,j))))
                        cnt += 1

            for i in range(cnt-1):
                a = q.get()
                b = q.get()
                q.put((a[0]+b[0], str("[" + a[1] + "," + b[1] + "]")))

            tree = eval(q.get()[1])
            return tree

        tree = make_tree(freq)
        code_lengths = np.zeros(freq.shape, dtype=int)

        depth = 0
        def get_depths(arr, code_lengths):
            global depth
            if type(arr[1]) == int and type(arr[0]) == int:
                code_lengths[arr[0], arr[1]] = min(depth, 16)
            else:
                depth += 1
                get_depths(arr[0], code_lengths)
                get_depths(arr[1], code_lengths)
                depth-=1

        get_depths(tree, code_lengths)
        return code_lengths
    
    code_lengths = get_code_lengths_from_huf(hufs[index], freq.shape[0], freq.shape[1])
#     code_lengths = get_code_lengths(freq)
    
    length_symbol_pairs = []
    
    m, n = freq.shape
    for i in range(m):
        for j in range(n):
            if code_lengths[i, j]:
                length_symbol_pairs.append((code_lengths[i, j], (i, j)))
    length_symbol_pairs.sort()
    
    def get_code(code, length):
        st = bin(code).replace("0b", "")[::-1]
        while len(st) < length:
            st += '0'
        return st[::-1]
    
    code = 0
    cur_len = 0
#     last = length_symbol_pairs[-1]
#     length_symbol_pairs[-1] = (last[0] + 1, last[1])
    for length, symbol in length_symbol_pairs:
        while cur_len < length:  #if there is no symbol with the current length
            cur_len += 1
            code *= 2
        codebook[symbol[0], symbol[1]] = get_code(code, length)
        code += 1
    
    print(length_symbol_pairs)
    return codebook, length_symbol_pairs

codes_dc_Y, sorted_symbols_dc_Y = get_codebook(dc_huffman_Y, 0)
codes_ac_Y, sorted_symbols_ac_Y = get_codebook(ac_huffman_Y, 1)
codes_dc_C, sorted_symbols_dc_C = get_codebook(dc_huffman_C, 2)
codes_ac_C, sorted_symbols_ac_C = get_codebook(ac_huffman_C, 3)
print(list(codes_dc_Y))
print(codes_ac_Y[15, 0])

def to_two_bytes(num):
    bh = bin(num).replace("0b", "")
    bh = bh[::-1]
    while len(bh) < 16:
        bh += '0'
    bh = bh[::-1]

    return [int(bh[:8], 2), int(bh[8:], 2)]

# Writing jpeg image in binary
jpeg_image = [0xff, 0xd8, 0xff, 0xe0, 0x00, 0x10]

header = [0x4a, 0x46, 0x49, 0x46, 0x00, 0x01, 0x01, 0x01, 0x00, 0x48, 0x00, 0x48, 0x00, 0x00]
jpeg_image.extend(header)

luminance_quantisation_table = [0xff, 0xdb, 0x00, 0x43, 0x00]
luminance_quantisation_table.extend([int(ele) for ele in spiral_traversal(Y_q)])
jpeg_image.extend(luminance_quantisation_table)

chrominance_quantisation_table = [0xff, 0xdb, 0x00, 0x43, 0x01]
chrominance_quantisation_table.extend([int(ele) for ele in spiral_traversal(C_q)])
jpeg_image.extend(chrominance_quantisation_table)

start_of_frame = [0xff, 0xc0, 0x00, 0x11, 0x08]
start_of_frame.extend(to_two_bytes(old_shape[0]))
start_of_frame.extend(to_two_bytes(old_shape[1]))
start_of_frame.extend([0x03, 0x01, 0x22, 0x00, 0x02, 0x11, 0x01, 0x03, 0x11, 0x01])
jpeg_image.extend(start_of_frame)

def parse_table(sorted_symbols, typ):
    table = [0xff, 0xc4]
    ln = 19
    symbols = []
    freqs = [0 for i in range(16)]
    
    for length, symbol in sorted_symbols:
#         print(length)
        freqs[length-1] += 1
        symbols.append(symbol[0] * 16 + symbol[1])
        ln += 1
#     print()
    
    table.extend(to_two_bytes(ln))
    table.append(typ)
    table.extend(freqs)
    table.extend(symbols)
    return table

jpeg_image.extend(parse_table(sorted_symbols_dc_Y, 0x00))
jpeg_image.extend(parse_table(sorted_symbols_ac_Y, 0x10))
jpeg_image.extend(parse_table(sorted_symbols_dc_C, 0x01))
jpeg_image.extend(parse_table(sorted_symbols_ac_C, 0x11))

start_of_scan = [0xff, 0xda, 0x00, 0x0c, 0x03, 0x01, 0x00, 0x02, 0x11, 0x03, 0x11, 0x00, 0x3f, 0x00]
jpeg_image.extend(start_of_scan)

def bit_rep(value):
    if value == 0:
        return ''
    else : 
        v = bin(abs(value)).replace("0b", '')
        if value>0 :
            return v
        else:
            return v.replace('0', 'a').replace('1','0').replace('a','1')

def encode_data(encoded_Y, encoded_Cb, encoded_Cr, codes_dc_Y, code_dc_C, codes_ac_Y, codes_ac_C):
    s = ''
    def encode_block(block, dc_codebook, ac_codebook):
        ans = ''
        dc_val = block[0][1]
        cat = get_cat(dc_val)
        ans+=(dc_codebook[0,cat]+bit_rep(dc_val))
        
        for i in range(1, len(block)):
            run, val = block[i]
#             print(run, get_cat(val))
            ans+=(ac_codebook[run, get_cat(val)]+bit_rep(val))
        return ans
    
    for i in range(len(encoded_Y)):
        s += encode_block(encoded_Y[i], codes_dc_Y, codes_ac_Y)
        if (i % 4 == 3):
            s += encode_block(encoded_Cb[i // 4], codes_dc_C, codes_ac_C)
            s += encode_block(encoded_Cr[i // 4], codes_dc_C, codes_ac_C)
        
    #pad with 0s
    a = len(s)%8
    s+='0'*a
    
    data = [int(s[i:i+8], 2) for i in range(0, len(s), 8)]
    fin_data = []
    for ele in data:
        fin_data.append(ele)
        if ele == 0xff:
            fin_data.append(0x00)
    return fin_data
    
jpeg_image.extend(encode_data(encoded_Y, encoded_Cb, encoded_Cr, codes_dc_Y, codes_dc_C, codes_ac_Y, codes_ac_C))

end_of_image = [0xff, 0xd9]
jpeg_image.extend(end_of_image)

try:
    with open("my_image.jpeg", 'wb') as f:
        for byte in jpeg_image:
            f.write(byte.to_bytes(1, byteorder='big'))
except Exception as e:
    print(e)

file = open('my_image.jpeg', 'rb')
data = file.read()
s = data.hex()

l = re.findall('..?', s)
while len(l) % 11:
    l = np.append(l, '0')
d = np.reshape(l, (-1,11))
d = [' '.join(ele) for ele in d]
print(len(d))
for ele in d:
    print(ele)

file.close()

[array([[  0, 339],
        [  0,   0]]) array([[0, 0],
                            [0, 0]]) array([[0, 0],
                                            [0, 0]]) array([[0, 0],
                                                            [0, 0]])]
[[   0 -680]
 [   0    0]]
[[0 0]
 [0 0]]
[[2 3 3 3 3 3 4 5 6 7 8 9 0 0 0 0]]
[(2, (0, 0)), (3, (0, 1)), (3, (0, 2)), (3, (0, 3)), (3, (0, 4)), (3, (0, 5)), (4, (0, 6)), (5, (0, 7)), (6, (0, 8)), (7, (0, 9)), (8, (0, 10)), (9, (0, 11))]
[[ 4  2  2  3  4  5  7  8 10 16 16  0  0  0  0  0]
 [ 0  4  5  7  9 11 16 16 16 16 16  0  0  0  0  0]
 [ 0  5  8 10 12 16 16 16 16 16 16  0  0  0  0  0]
 [ 0  6  9 12 16 16 16 16 16 16 16  0  0  0  0  0]
 [ 0  6 10 16 16 16 16 16 16 16 16  0  0  0  0  0]
 [ 0  7 11 16 16 16 16 16 16 16 16  0  0  0  0  0]
 [ 0  7 12 16 16 16 16 16 16 16 16  0  0  0  0  0]
 [ 0  8 12 16 16 16 16 16 16 16 16  0  0  0  0  0]
 [ 0  9 15 16 16 16 16 16 16 16 16  0  0  0  0  0]
 [ 0  9 16 16 16 16 16 16 16 16 16  0  0  0  0  0]
 [ 0  9